In [1]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import getpass

import matplotlib.pyplot as plt
import seaborn as sns

https://archive.ics.uci.edu/dataset/502/online+retail+ii

As i reviewed the data further in sql, I decided I will only focus on sales first.

suspected returns make up less than 3% of all the transactions.

In [24]:
db = 'retail'
password = getpass.getpass("Please enter the SQL database password: ")
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+db
engine = create_engine(connection_string)

Please enter the SQL database password: ········


In [35]:
with engine.connect() as connection:
    query1 = text(
        """CREATE TEMPORARY TABLE lifetime_value
With cust_sales AS(
					SELECT cust_id, 
					ROUND(SUM(total_price)) as total_revenue,
					MIN(date) as first_purchase,
					MAX(date) as last_purchase,
					DATEDIFF('2011-01-01 09:26:00', MIN(date)) AS lifetime,
					COUNT(DISTINCT(date)) as frequency
					FROM sales
					GROUP BY cust_id
                    )
SELECT cust_id, 
		total_revenue,
		first_purchase,
		last_purchase,
		lifetime,
		frequency,
		ROUND((total_revenue/frequency)*4.4458) as CV,
		ROUND(((total_revenue/frequency)*4.4458)*134.2622) as CLV
FROM cust_sales;""")
    result = connection.execute(query1)

OperationalError: (pymysql.err.OperationalError) (1050, "Table 'lifetime_value' already exists")
[SQL: CREATE TEMPORARY TABLE lifetime_value
With cust_sales AS(
					SELECT cust_id, 
					ROUND(SUM(total_price)) as total_revenue,
					MIN(date) as first_purchase,
					MAX(date) as last_purchase,
					DATEDIFF('2011-01-01 09:26:00', MIN(date)) AS lifetime,
					COUNT(DISTINCT(date)) as frequency
					FROM sales
					GROUP BY cust_id
                    )
SELECT cust_id, 
		total_revenue,
		first_purchase,
		last_purchase,
		lifetime,
		frequency,
		ROUND((total_revenue/frequency)*4.4458) as CV,
		ROUND(((total_revenue/frequency)*4.4458)*134.2622) as CLV
FROM cust_sales;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [45]:
with engine.connect() as connection:
    query2 = text(
        """SELECT sales.*,
            total_revenue,
            first_purchase,
            last_purchase,
            lifetime,
            frequency, 
            lifetime_value.CV,
            lifetime_value.CLV
            FROM sales
            JOIN lifetime_value
            ON lifetime_value.cust_id = sales.cust_id;"""
                )
    result = connection.execute(query2)
    rows = result.fetchall()
    
df = pd.DataFrame(rows, columns=result.keys())

# Display the DataFrame
print(df.head())

Empty DataFrame
Columns: [invoice, stockcode, product, quantity, date, price, cust_id, country, month, year, total_price, total_revenue, first_purchase, last_purchase, lifetime, frequency, CV, CLV]
Index: []


In [42]:
df = pd.DataFrame(rows, columns=result.keys())

In [43]:
df

,invoice,stockcode,product,quantity,date,price,cust_id,country,month,year,total_price,total_revenue,first_purchase,last_purchase,lifetime,frequency,CV,CLV
